In [2]:
import pandas as pd 
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

In [4]:
df = pd.read_csv('TA_restaurants_curated.csv')
df.head()

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959
3,3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",/Restaurant_Review-g188590-d1239229-Reviews-Vi...,d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",/Restaurant_Review-g188590-d6864170-Reviews-Li...,d6864170


In [46]:
# encode the price range as a number from 1 to 4
df['Price Range'] = df['Price Range'].replace('$', 1)
df['Price Range'] = df['Price Range'].replace('$$', 2)
df['Price Range'] = df['Price Range'].replace('$$ - $$$', 3)
df['Price Range'] = df['Price Range'].replace('$$$$', 4)

df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: eval(x) if pd.notnull(x) else [])

df.head()

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"[French, Dutch, European]",1.0,5.0,3.0,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"[Dutch, European, Vegetarian Friendly, Gluten ...",2.0,4.5,4.0,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,2,La Rive,Amsterdam,"[Mediterranean, French, International, Europea...",3.0,4.5,4.0,567.0,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959
3,3,Vinkeles,Amsterdam,"[French, European, International, Contemporary...",4.0,5.0,4.0,564.0,"[['True five star dinner', 'A superb evening o...",/Restaurant_Review-g188590-d1239229-Reviews-Vi...,d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"[Dutch, European, International, Vegetarian Fr...",5.0,4.5,4.0,316.0,"[['Best meal.... EVER', 'super food experience...",/Restaurant_Review-g188590-d6864170-Reviews-Li...,d6864170


In [51]:
def query_restaurants(dataframe, query_params, default_city=None, max_results=10):
    """
    Query the restaurant dataset based on various tags.
    
    :param dataframe: Pandas DataFrame containing the restaurant data.
    :param query_params: Dictionary with keys as tags ('Cuisine', 'City', 'Name') 
                         and values as strings to query.
    :param default_city: Default city to use if no city is specified in query_params.
    :param max_results: Maximum number of results to return if the city is not specified.
    :return: Filtered DataFrame based on the query parameters.
    """
    filtered_df = dataframe

    # if query params isnt about cuisine, city, or name, return empty dataframe
    if not any(key in query_params for key in ['Cuisine Style', 'City', 'Name']):
        print('No results found for the given query.')

    # Apply city filter
    if 'City' in query_params and query_params['City']:
        filtered_df = filtered_df[filtered_df['City'].str.contains(query_params['City'], case=False, na=False)]
    elif default_city:
        filtered_df = filtered_df[filtered_df['City'] == default_city]

    # Apply other filters
    if 'Name' in query_params:
        filtered_df = filtered_df[filtered_df['Name'].str.contains(query_params['Name'], case=False, na=False)]

    if 'Cuisine Style' in query_params:
        # filter the restaurants that have this cuisine in their Cuisine Style list
        filtered_df = filtered_df[filtered_df['Cuisine Style'].apply(lambda x: query_params['Cuisine Style'] in x)]
    else:
        # If no cuisine is specified, sort by rating
        filtered_df = filtered_df.sort_values(by='Rating', ascending=False)
    
    # if filtered_df is the same as the original df, then no filters were applied and no results were found
    print(filtered_df.shape[0])
    print(dataframe.shape[0])
    if filtered_df.shape[0] == dataframe.shape[0] or filtered_df.shape[0] == 0:
        print('No results found for the given query.')
    else:
        print('Found {} results.'.format(filtered_df.shape[0]))

    # return at most 3 results
    return filtered_df.head(3)

In [61]:
query_params = {'Cuisine Style': 'Greek', 'City': 'Madrid'}

query_restaurants(df, query_params)

24
125527
Found 24 results.


,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
67896,517,Greek and Shop,Madrid,"[Fast Food, Mediterranean, European, Greek, Ve...",518.0,4.0,1.0,275.0,"[['tasty food, friendly service', 'Handling fo...",/Restaurant_Review-g187514-d4745803-Reviews-Gr...,d4745803
67928,549,Greek and Shop,Madrid,"[Fast Food, Mediterranean, European, Greek, Ve...",550.0,4.0,1.0,275.0,"[['tasty food, friendly service', 'Handling fo...",/Restaurant_Review-g187514-d4745803-Reviews-Gr...,d4745803
67981,602,Dionisos Huertas,Madrid,"[Mediterranean, European, Greek, Vegetarian Fr...",603.0,4.0,3.0,442.0,"[['Griego rico', 'Try the olives'], ['11/24/20...",/Restaurant_Review-g187514-d3718338-Reviews-Di...,d3718338
